In [1]:
import gc
import os
import sys
from multiprocessing import Process

import numpy as np
import tensorflow as tf

from tensorflow.keras import backend as K

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sys.path.insert(0, r"../utilities/")
sys.path.insert(0, r"../Seq2Seq/")
sys.path.insert(0, r"../Seq2SeqAttention/")
sys.path.insert(0, r"../Transformer/")

from utils import *

In [2]:
from Seq2SeqTrainer import Seq2SeqTrainer
from Seq2SeqAttentionTrainer import Seq2SeqAttentionTrainer
from TransformerTrainer import TransformerTrainer

In [3]:
def makePlots(losses, accuracy, name):
    train_losses, test_losses = losses 
    train_accuracyVec, test_accuracyVec = accuracy

    import matplotlib.pyplot as plt
    fig = plt.figure()
    fig_plot = fig.add_subplot()
    fig_plot.plot(train_losses, label="train_loss")
    fig_plot.plot(test_losses, label="test_loss")
    fig_plot.legend(loc="upper right")
    fig_plot.set_xlabel("epoch")
    fig_plot.set_ylabel("loss")
    fig_plot.grid(linestyle="--")
    fig.savefig("losses_plot_" + name +  ".png")
    fig.show()

    fig = plt.figure()
    fig_plot = fig.add_subplot()
    fig_plot.plot(train_accuracyVec, label="train_accuracy")
    fig_plot.plot(test_accuracyVec, label="test_accuracy")
    fig_plot.legend(loc="lower right")
    fig_plot.set_xlabel("epoch")
    fig_plot.set_ylabel("accuracy")
    fig_plot.grid(linestyle="--")
    fig.savefig("accuracy_plot.png")
    fig.show()

In [4]:
data_dir = "../data/"
# reading data

data = read_data(os.path.join(data_dir, "fra-eng"), "fra.txt")
en_lines, fr_lines = list(zip(*data))
en_lines_raw, fr_lines_raw = shuffle(en_lines, fr_lines)

en_lines = en_lines_raw[:40000]
fr_lines = fr_lines_raw[:40000]

en_lines = [normalize(line) for line in en_lines]
fr_lines = [normalize(line) for line in fr_lines]

en_train, en_test, fr_train, fr_test = train_test_split(en_lines, fr_lines, shuffle=True, test_size=0.1)

en_lines = en_test
fr_lines = fr_test

fr_train_in = ['<start> ' + line for line in fr_train]
fr_train_out = [line + ' <end>' for line in fr_train]

fr_test_in = ['<start> ' + line for line in fr_test]
fr_test_out = [line + ' <end>' for line in fr_test]

fr_tokenizer = Tokenizer(filters='')
en_tokenizer = Tokenizer(filters='')

input_data = [fr_train_in, fr_train_out, fr_test_in, fr_test_out, fr_test, fr_train]
fr_train_in, fr_train_out, fr_test_in, fr_test_out, fr_test, fr_train = tokenizeInput(input_data,
                                                                                      fr_tokenizer)
input_data = [en_train, en_test]
en_train, en_test = tokenizeInput(input_data, en_tokenizer)

en_vocab_size = len(en_tokenizer.word_index)+1
fr_vocab_size = len(fr_tokenizer.word_index)+1
print("en_vocab {}\nfr_vocab {}" .format(en_vocab_size, fr_vocab_size))

reading data from  ../data/fra-eng/fra.txt
en_vocab 8397
fr_vocab 13499


In [5]:
prediction_idx = np.random.randint(low=40000, high=len(en_lines_raw), size=10)
print("TEST_TEXTS")
test_text = [(en_lines_raw[idx], fr_lines_raw[idx]) for idx in prediction_idx]
for (en,fr) in test_text:
    print(en, " - ", fr)

TEST_TEXTS
I'm not feeling particularly hungry.  -  Je ne ressens pas particulièrement la faim.
She's a quiet person.  -  Elle est d'un naturel calme.
We all hate Tom.  -  Nous détestons tous Tom.
Tom wouldn't really do that, would he?  -  Tom ne ferait pas vraiment cela, n'est-ce pas ?
I'm in love with you and I want to marry you.  -  Je suis amoureux de toi et je veux t'épouser.
Where are your gloves?  -  Où sont tes gants ?
You two are really kind.  -  Vous êtes vraiment gentils tous les deux.
What you make is small potatoes compared to the boss's salary.  -  Ce que tu gagnes est de la roupie de sansonnet, comparé au salaire du patron.
Mama cried.  -  Maman pleurait.
How did you come by this painting?  -  Comment êtes-vous entré en possession de cette toile ?


In [6]:
LSTM_SIZE = 512
EMBEDDING_SIZE = 256
BATCH_SIZE= 64
EPOCHS = 20

In [7]:
def Seq2SeqPredictions():
    trainer = Seq2SeqTrainer(BATCH_SIZE, LSTM_SIZE, EMBEDDING_SIZE, predict_every=20)
    losses, accuracy = trainer.train([en_train, fr_train_in, fr_train_out], [en_test, fr_test_in, fr_test_out], [en_lines, fr_lines], [en_tokenizer, fr_tokenizer], EPOCHS)
    makePlots(losses, accuracy, "Seq2Seq")
    for (en_text, fr_text) in test_text:
        trainer.predict(en_text, fr_text)

def Seq2SeqAttentionPredictions():
    trainer = Seq2SeqAttentionTrainer(BATCH_SIZE, LSTM_SIZE, EMBEDDING_SIZE, predict_every=20)
    losses, accuracy = trainer.train([en_train, fr_train_in, fr_train_out], [en_test, fr_test_in, fr_test_out], [en_lines, fr_lines], [en_tokenizer, fr_tokenizer], EPOCHS, "concat")
    makePlots(losses, accuracy, "Seq2SeqAttention")
    for (en_text, fr_text) in test_text:
        trainer.predict(en_text, fr_text, print_prediction=True)

def TransformerPredictions():
    BATCH_SIZE = 64
    num_layers = 4 # 6
    d_model = 256 # 512
    dff = 512  # 2048
    num_heads = 8
    trainer = TransformerTrainer(BATCH_SIZE, num_layers, d_model, dff, num_heads, predict_every=20)
    losses, accuracy= trainer.train([en_train, fr_train_in, fr_train_out], [en_test, fr_test_in, fr_test_out], [en_lines, fr_lines], [en_tokenizer, fr_tokenizer], EPOCHS)
    makePlots(losses, accuracy, "Transformer")
    for (en_text, fr_text) in test_text:
        trainer.predict(en_text, fr_text)

In [8]:
p = Process(target=Seq2SeqPredictions, args=())
p.start()
p.join()

en_vocab 8397
fr_vocab 13499
Number of devices: 4
creating dataset...
dataset created
input :  Were you born there ?
output:  Etes-vous ne la-bas ?
training from scratch
starting training with 20 epochs with prediction each 20 epoch
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0

In [9]:
p = Process(target=Seq2SeqAttentionPredictions, args=())
p.start()
p.join()

en_vocab 8397
fr_vocab 13499
Number of devices: 4
creating dataset...
dataset created
input :  Were you born there ?
output:  Etes-vous ne la-bas ?
training from scratch
starting training with 20 epochs with prediction each 20 epoch
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:

----------------------------PREDICTION----------------------------
       En sentence How did you come by this painting? 
       Predicted:  comment es-tu a toi contre ca . 
       Should be:  Comment êtes-vous entré en possession de cette toile ? 
--------------------------END PREDICTION--------------------------


In [10]:
p = Process(target=TransformerPredictions, args=())
p.start()
p.join()

en_vocab 8397
fr_vocab 13499
Number of devices: 4
creating dataset...
input :  Were you born there ?
output:  Etes-vous ne la-bas ?
training from scratch
INFO:tensorflow:batch_all_reduce: 170 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/devi